This is an example Jupyter notebook on using GELATO.
----------------

First let's import the packages we will need.

In [ ]:
# Import packages
import gelato
import numpy as np
%matplotlib inline
import matplotlib
mpl.rcParams['font.size'] = 25
from matplotlib import pyplot # For plotting
# For loading in data
from astropy.io import fits
from astropy.table import Table 

We're going to be running our example on the SDSS spectrum, spec-0280-51612-0117.fits, which has a redshift of 0.245. 

In [ ]:
# Let's load the spectrum
path_spec = 'Spectra/spec-0280-51612-0117.fits'
spectrum = Table.read(path_spec)

# Start with inverse variance
ivar = spectrum['ivar']
good = ivar > 0 # GELATO only looks at points with nonzero weights

# Finally, let's load in the data
wavl = 10**spectrum['loglam'][good]
flux = spectrum['flux'][good]
ivar = ivar[good]
args = (wavl,flux,ivar) # These will be useful later

Let's go ahead and plot our spectrum to get an idea of what we're dealing with.

In [ ]:
# Create figure
fig, ax = pyplot.subplots(figsize=(15,7))

# Plot Spectrum
sig = 3/np.sqrt(ivar) # 3 Sigma boundary
ax.fill_between(wavl,flux-sig,flux+sig,color='gray')
ax.step(wavl,flux,where='mid',c='k',lw=0.5)

# Axis limits
ax.set(xlim=[wavl.min(),wavl.max()],ylim=[0,flux.max()])

# Axis labels
ax.set(xlabel=r'Obs. Wavelength [\AA]',ylabel=r'$F_\lambda$')

# Show figure
pyplot.show()

The main gelato function takes three inputs.
* The path to the parameters file or the parameters dictionary.
* The path to the spectrum.
* The redshift of the spectrum.

We already have the last two, and we need to take a little precaution with the first.
The main gelato function will only return the final model if the code is being run without multiprocessing (as the return statement can break Python multiprocessing). So we can either change the Parameters JSON file, or edit the parameters dictionary. 

In [ ]:
# Path to the parameters file
path_params = './ExampleParameters.json'

# Create Parameters dictionary
params = gelato.ConstructParams.construct(path_params)

# Set to not multiprocessing
params['NProcess'] = 1

We are now ready to run GELATO. Note, before you do this, ensure the results directory exists, either by running the Example from the README file or creating it. It will return the final callable model, however it won't be used in this notebook. 

In [ ]:
model = gelato.gelato(params,path_spec,0.245)

The results have been saved to the "Results/" Directory. Let's go ahead and load them in. We will print all extensions on the folder.

In [ ]:
# Load in results
results = fits.open('Results/spec-0280-51612-0117-results.fits')

# Print FITS extensions
results.info()

We have two FITS extensions, SUMMARY and PARAMS. They are described in more detail in the README File but let's play around with them directly. Let's go ahead and take a look inside the SUMMARY extension. As we can see, it is a binary FITS Table.

In [ ]:
summary = Table(results['SUMMARY'].data)
summary

In this table, we have the original spectrum along with the various model components, we can go ahead and plot them.

In [ ]:
# Create figure
fig, ax = pyplot.subplots(figsize=(15,7))

# Plot Spectrum
ax.fill_between(wavl,flux-sig,flux+sig,color='gray')
ax.step(wavl,flux,where='mid',c='k',lw=0.5,label='Data')
ax.step(10**summary['loglam'],summary['MODEL'],where='mid',c='r',label='Total Model')
ax.step(10**summary['loglam'],summary['SSP'],where='mid',c='g',label='SSP Cont.')
ax.step(10**summary['loglam'],summary['PL'],where='mid',c='b',label='Power-Law Cont.')
ax.step(10**summary['loglam'],summary['LINE'],where='mid',c='y',label='Emission Lines')
ax.legend()

# Axis limits
ax.set(xlim=[wavl.min(),wavl.max()],ylim=[0,flux.max()])

# Axis labels
ax.set(xlabel=r'Obs. Wavelength [\AA]',ylabel=r'$F_\lambda$')

# Show figure
pyplot.show()

Looks great! You can see an example of the GELATO generated plots in the results folder, but this will let you incorporate GELATO fits easily into your own work. Let's go ahead and take a look at the PARAMS extension. This is a much larger table! It's made up of the parameters from each bootstrap iteration. 

In [ ]:
# Open Parameters extension
params = Table(results['PARAMS'].data)
print(params)
params.colnames

Here we can see all of the fitted model paramters, it's certainly a handful! A quick note, many parameters here are tied together, reducing the degrees of freedom. It's also worth noting the SSP Continuum Redshift and the PL Continuum Scale are not fitted, and so are constant throughout all the bootstraps. Let's go ahead and throw this object onto a BPT diagram!

In [ ]:
# Get BPT line fluxes
oiii = params['AGN_[OIII]_5006.84_Flux']
nii = params['AGN_[NII]_6583.45_Flux']
ha = params['Balmer_HI_4861.28_Flux']
hb = params['Balmer_HI_6562.79_Flux']

# Create figure
fig, ax = pyplot.subplots(figsize=(10,10))

# Kewley+ Line
x = np.logspace(-1.5,0.05,100)
y = 10**(0.61/(np.log10(x) - 0.05) + 1.3)
ax.plot(x,y,color='gray',ls='--',label='Kauffman+03')

# Kauffman+ Line
x = np.logspace(-1.5,0.47,100)
y = 10**(0.61/(np.log10(x) - 0.47) + 1.18)
ax.plot(x,y,color='gray',ls='-',label='Kewley+01')

# Plot BPT
ax.scatter(nii/ha,oiii/hb,color='k',label='Bootstraps',edgecolors='none',alpha=0.1)
x,y,xerr,yerr = np.median(nii/ha),np.median(oiii/hb),np.std(nii/ha),np.std(oiii/hb)
ax.errorbar(x,y,xerr=xerr,yerr=yerr,color='r',label='Average')
ax.legend()

# Axis limits
ax.set(xlim=[1e-1,1e1],ylim=[1e-1,1e1])

# Axis labels
ax.set(xlabel=r'[NII]/H$\alpha$',ylabel=r'[OIII]/H$\beta$')

# Axis scale
ax.set(yscale='log',xscale='log')

# Show figure
pyplot.show()

And that's all for this short IPython notebook! GELATO is designed to be run through the wrapper scripts, but hopefully this helps if you want to run in an IPython notebook or to help you understand the GELATO output.